🔗 TASK 4 – MERGE DỮ LIỆU (CHUẨN & SẠCH)

🔹 1. Import thư viện & đọc dữ liệu sạch

In [ ]:
import pandas as pd

df_candidate = pd.read_csv("../data_clean/candidate_profile_clean.csv")
df_interview = pd.read_csv("../data_clean/interview_score_clean.csv")
df_salary = pd.read_csv("../data_clean/salary_offer_clean.csv")


2. Merge candidate_profile với interview_summary

In [2]:
score_summary = (
    df_interview
    .groupby("candidate_id")["score"]
    .agg(
        avg_score="mean",
        max_score="max",
        min_score="min",
        interview_rounds="count"
    )
    .reset_index()
)


In [3]:
df_merge_1 = df_candidate.merge(
    score_summary,
    on="candidate_id",
    how="left"
)


📝 Nhận xét:

Sau bước này, mỗi ứng viên có thêm các thông tin tổng hợp về điểm phỏng vấn như điểm trung bình, điểm cao nhất và số vòng tham gia.

🔹 3. Merge thêm bảng salary_offer

In [10]:
df_final = df_merge_1.merge(
    df_salary,
    on="candidate_id",
    how="left"
)

df_final.head()


,candidate_id,full_name,experience_years,avg_score,max_score,min_score,interview_rounds,offer_salary,position
0,UV100,Hoang Van E,3.0,9.000000,9.0,9.0,1,15000000.0,Data Analyst
1,UV101,Le Van C,NaN,7.666667,9.0,7.0,3,20000000.0,Data Analyst
2,UV102,Le Van C,NaN,5.000000,5.0,5.0,1,25000000.0,Data Analyst
3,UV103,Tran Thi B,NaN,6.000000,6.0,6.0,2,15000000.0,Python Developer
4,UV104,Pham Thi D,NaN,6.000000,7.0,5.0,3,15000000.0,Data Analyst


🔹 4. Phát hiện mismatch dữ liệu

🔸 4.1 Ứng viên có phỏng vấn nhưng không có lương đề nghị

In [11]:
missing_salary = df_final[
    df_final["offer_salary"].isna()
]

missing_salary[["candidate_id", "full_name", "avg_score"]].head()


,candidate_id,full_name,avg_score
5,UV105,Tran Thi B,7.333333
13,UV113,Hoang Van E,7.666667
14,UV114,Nguyen Van A,7.000000
18,UV118,Tran Thi B,7.000000
23,UV123,Tran Thi B,6.333333


📝 Nhận xét:Một số ứng viên đã tham gia phỏng vấn nhưng chưa có thông tin lương đề nghị, có thể do chưa vào vòng offer.

🔸 4.2 Ứng viên có lương đề nghị nhưng chưa có điểm phỏng vấn

In [12]:
missing_interview = df_final[
    df_final["avg_score"].isna()
]

missing_interview[["candidate_id", "full_name", "offer_salary"]].head()


,candidate_id,full_name,offer_salary
11,UV111,Tran Thi B,20000000.0
17,UV117,Le Van C,15000000.0


📝 Nhận xét:Các trường hợp này có thể do dữ liệu phỏng vấn chưa được cập nhật đầy đủ.

🔹 5. Kiểm tra tổng quan dữ liệu sau merge

In [13]:
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   candidate_id      25 non-null     object 
 1   full_name         25 non-null     object 
 2   experience_years  15 non-null     float64
 3   avg_score         23 non-null     float64
 4   max_score         23 non-null     float64
 5   min_score         23 non-null     float64
 6   interview_rounds  25 non-null     int64  
 7   offer_salary      20 non-null     float64
 8   position          25 non-null     object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.9+ KB


In [14]:
df_final.isna().sum()


candidate_id         0
full_name            0
experience_years    10
avg_score            2
max_score            2
min_score            2
interview_rounds     0
offer_salary         5
position             0
dtype: int64

🔹 6. Xuất file dữ liệu hoàn chỉnh

In [15]:
import os

# Tạo thư mục data_clean nếu chưa tồn tại
os.makedirs("../data_clean", exist_ok=True)

# Xuất file dữ liệu HR hoàn chỉnh
df_final.to_csv(
    "../data_clean/hr_full_data.csv",
    index=False
)

print("Đã xuất file hr_full_data.csv")


Đã xuất file hr_full_data.csv
